# Model Complexity Analysis Tool

目前支持 Conv2D / DepthwiseConv2D / SeparableConv2D 的时间复杂度统计。

In [1]:
from keras import backend as K
import numpy as np

def model_analysis(model, log_level=1):
    ops = 0
    n_std_conv = 0
    n_dw_conv = 0
    n_pw_conv = 0
    for l in model.layers:
        layer_type = l.__class__.__name__

        if layer_type == 'Conv2D':
            output_shape = int(l.output.shape[1])
            input_channel = int(l.input.shape[-1])
            output_channel = int(l.output.shape[-1])
            k_size = l.kernel_size[0]
            ops += (output_shape ** 2) * (k_size ** 2) * input_channel * output_channel    # STD CONV
            n_std_conv += 1
            if k_size == 1:
                n_pw_conv += 1
            if log_level > 1:
                print(output_shape, '*', output_shape, '*', k_size, '*', k_size, '*', input_channel, '*', output_channel)

        elif layer_type == 'DepthwiseConv2D':
            output_shape = int(l.output.shape[1])
            input_channel = int(l.input.shape[-1])
            k_size = l.kernel_size[0]
            ops += (output_shape ** 2) * (k_size ** 2) * input_channel    # DW
            n_dw_conv += 1
            if log_level > 1:
                print(output_shape, '*', output_shape, '*', k_size, '*', k_size, '*', input_channel)
                
        elif layer_type == 'SeparableConv2D':
            output_shape = int(l.output.shape[1])
            input_channel = int(l.input.shape[-1])
            k_size = l.kernel_size[0]
            n_dw_conv += 1
            n_std_conv += 1
            n_pw_conv += 1
            ops += (output_shape ** 2) * (k_size ** 2) * input_channel    # DW
            ops += (output_shape ** 2) * input_channel * output_channel   # PW
            
        elif layer_type == 'Add':
            output_shape = l.output_shape[1:]
            ops += np.prod(output_shape)
            if log_level > 1:
                print(output_shape[0], '*', output_shape[1], '*', output_shape[2])
                
    trainable = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))
    
    if log_level > 0:
        print('\n- Model Name \t\t= %s' % model.name)
        print('- Convolution Layers \t= %d (%d are Depth-wise, %d are Point-wise)' % (n_std_conv + n_dw_conv, n_dw_conv, n_pw_conv))
        print('- Time Complexity \t= %.1f MFLOPS' % (ops / 10 ** 6))
        print('- Space Complexity \t= %.1f MParams\n' % ((trainable + non_trainable) / 10 **6))

Using TensorFlow backend.


## Example

In [2]:
from keras.applications import MobileNet, InceptionV3, Xception, InceptionResNetV2, VGG16, VGG19, ResNet50

In [4]:
mobilenet = MobileNet(alpha=1, weights=None)
inceptionv3 = InceptionV3(input_shape=(299, 299, 3))
xception = Xception(input_shape=(299, 299, 3))
inceptionresnet = InceptionResNetV2(input_shape=(299, 299, 3))

model_analysis(mobilenet)
model_analysis(inceptionv3)
model_analysis(xception)
model_analysis(inceptionresnet)


- Model Name 		= mobilenet_1.00_224
- Convolution Layers 	= 28 (13 are Depth-wise, 14 are Point-wise)
- Time Complexity 	= 568.7 MFLOPS
- Space Complexity 	= 4.3 MParams


- Model Name 		= inception_v3
- Convolution Layers 	= 94 (0 are Depth-wise, 57 are Point-wise)
- Time Complexity 	= 9697.5 MFLOPS
- Space Complexity 	= 23.9 MParams


- Model Name 		= xception
- Convolution Layers 	= 74 (34 are Depth-wise, 38 are Point-wise)
- Time Complexity 	= 6900.9 MFLOPS
- Space Complexity 	= 22.9 MParams


- Model Name 		= inception_resnet_v2
- Convolution Layers 	= 244 (0 are Depth-wise, 170 are Point-wise)
- Time Complexity 	= 20066.7 MFLOPS
- Space Complexity 	= 55.9 MParams



In [3]:
vgg16 = VGG16()
vgg19 = VGG19()
resnet50 = ResNet50()

model_analysis(vgg16)
model_analysis(vgg19)
model_analysis(resnet50)


- Model Name 		= vgg16
- Convolution Layers 	= 13 (0 are Depth-wise, 0 are Point-wise)
- Time Complexity 	= 15346.6 MFLOPS
- Space Complexity 	= 138.4 MParams


- Model Name 		= vgg19
- Convolution Layers 	= 16 (0 are Depth-wise, 0 are Point-wise)
- Time Complexity 	= 19508.4 MFLOPS
- Space Complexity 	= 143.7 MParams


- Model Name 		= resnet50
- Convolution Layers 	= 53 (0 are Depth-wise, 36 are Point-wise)
- Time Complexity 	= 3837.7 MFLOPS
- Space Complexity 	= 25.6 MParams



In [5]:
model_analysis(mobilenet, 2)

112 * 112 * 3 * 3 * 3 * 32
112 * 112 * 3 * 3 * 32
112 * 112 * 1 * 1 * 32 * 64
56 * 56 * 3 * 3 * 64
56 * 56 * 1 * 1 * 64 * 128
56 * 56 * 3 * 3 * 128
56 * 56 * 1 * 1 * 128 * 128
28 * 28 * 3 * 3 * 128
28 * 28 * 1 * 1 * 128 * 256
28 * 28 * 3 * 3 * 256
28 * 28 * 1 * 1 * 256 * 256
14 * 14 * 3 * 3 * 256
14 * 14 * 1 * 1 * 256 * 512
14 * 14 * 3 * 3 * 512
14 * 14 * 1 * 1 * 512 * 512
14 * 14 * 3 * 3 * 512
14 * 14 * 1 * 1 * 512 * 512
14 * 14 * 3 * 3 * 512
14 * 14 * 1 * 1 * 512 * 512
14 * 14 * 3 * 3 * 512
14 * 14 * 1 * 1 * 512 * 512
14 * 14 * 3 * 3 * 512
14 * 14 * 1 * 1 * 512 * 512
7 * 7 * 3 * 3 * 512
7 * 7 * 1 * 1 * 512 * 1024
7 * 7 * 3 * 3 * 1024
7 * 7 * 1 * 1 * 1024 * 1024
1 * 1 * 1 * 1 * 1024 * 1000

- Model Name 		= mobilenet_1.00_224
- Convolution Layers 	= 28 (13 are Depth-wise)
- Time Complexity 	= 568.7 MFLOPS
- Space Complexity 	= 4.3 MParams

